In [ ]:
!pip install crewai groq --progress-bar off
# !sudo apt install tesseract-ocr --progress-bar off
# !sudo apt install libtesseract-dev --progress-bar off
# !pip install pytesseract --progress-bar off

In [ ]:
!sudo apt install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
import os
os.environ['GROQ_API_KEY'] = "your groq api key"


In [ ]:
from crewai import Agent, Task, Crew, LLM
import io
from PIL import Image
import pytesseract

In [ ]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union
import numpy as np
np.float_ = np.float64
import pandas as pd
from crewai import Agent, Crew, Process, Task
from google.colab import userdata
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult


import io
from PIL import Image
import pytesseract


In [ ]:
llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=1,
    max_completion_tokens=1024,
    top_p=0.9,
    stop=None,
    stream=False,
)

import shutil


In [ ]:
def run_tesseract(img):

    # img = Image.open(img)
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='JPEG')
    img_byte_arr = img_byte_arr.getvalue()
    img = Image.open(io.BytesIO(img_byte_arr))

    pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
    text = pytesseract.image_to_string(img)
    # del state['the_image']
    return {text}
img = Image.open("/content/FJPCYCdaMAMT1O3.jpg").convert("RGB")
text = run_tesseract(img)

In [ ]:
from crewai import Agent, Task, Crew, LLM

# Initialize Large Language Model (LLM) of your choice (see all models on our Models page)
# llm = LLM(model="groq/llama-3.1-70b-versatile")

# Create your CrewAI agents with role, main goal/objective, and backstory/personality
summarizer = Agent(
    role='Receipt Price Extractor', # Agent's job title/function
    goal='Take out the total price out of the text and remove all currency signs', # Agent's main objective
    backstory='Technical price extractor who excels at identifying price from receipts', # Agent's background/expertise
    llm=llm, # LLM that powers your agent
    verbose=True # Show agent's thought process as it completes its task
)

translator = Agent(
    role='Receipt categorizer',
    goal='categorizer Receipts to different types like fuel, travel, hotel, food etc.',
    backstory='Technical Receipt categorizer specializing in identifying type of receipts',
    llm=llm,
    verbose=True
)

# Define your agents' tasks
summary_task = Task(
    description=f"Extract the price from the text: {text}",
    expected_output="Return me just the price as a number",
    agent=summarizer # Agent assigned to task
)

translation_task = Task(
    description=f"Extract the receipt type from the text:  {text}",
    expected_output="just categorize the text in one word",
    agent=translator,
    dependencies=[summary_task] # Must run after the summary task
)

# Create crew to manage agents and task workflow
crew = Crew(
    agents=[summarizer, translator], # Agents to include in your crew
    tasks=[summary_task, translation_task], # Tasks in execution order
    verbose=True
)

result = crew.kickoff()
print(result)



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 94ca4752-4bf0-4fe5-bb50-f623c47b7cfe                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 5584aa41-5146-4218-919f-51bef5d6cd70
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 5584aa41-5146-4218-919f-51bef5d6cd70
       Status: Executing Task...
    └── 🤖 Agent: Receipt Price Extractor
            Status: In Progress

# Agent: Receipt Price Extractor
## Task: Extract the price from the text: {'APOORVA DELICACTES\nPure Veg\nDacar(W) ,Mumbat\nweeseenennne TAX INVOICE -~-----------\n\nDate: 16/01/22 Bill No. : dl\n\n(ity Rate Amount\n\n \n\nEDU WADA 1 65 65\nSAO DOSA i ioe\nMSL. DOSA 1 80 80\nONION RAVA DOSA 2 80 160\nCHEESE TOAST $/W 2 115 (230\nsub Total : 605-00\nSGST. 27AAFHK4833E 12C @2.5% + Le\nDGST.2TAAFHKAB33E 1ZC 2.5% : 15.12\nsft lotal 635\n\nFSSAI NO - 11516004000575 (08:13 PM)\nE.80.E. Thank You Visit Again\n\nLXXKK = 7\n\n \n  \n\nPLEASE CHECK TH: BILL BEFORE MAKING\nTHE PAYMENT, NO COMPLAINTS WILL” 36>\nENTERTAIN THEREAFTER ,\n\nOS\n// SRI GANESHA NAMAH\nTake Away\n\x0c'}


🚀 Crew: crew
└── 📋 Task: 5584aa41-5146-4218-919f-51bef5d6cd70
       Status: Executing Task...
    └── 🤖 Agent: Receipt Price Extractor
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 5584aa41-5146-4218-919f-51bef5d6cd70
│      Assigned to: Receipt Price Extractor
│      Status: ✅ Completed
│   └── 🤖 Agent: Receipt Price Extractor
│           Status: ✅ Completed
└── 📋 Task: 9bd40b6b-2d33-4baf-a93a-82dbdb6585cb
       Status: Executing Task...
    └── 🤖 Agent: Receipt categorizer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 5584aa41-5146-4218-919f-51bef5d6cd70
│      Assigned to: Receipt Price Extractor
│      Status: ✅ Completed
│   └── 🤖 Agent: Receipt Price Extractor
│           Status: ✅ Completed
└── 📋 Task: 9bd40b6b-2d33-4baf-a93a-82dbdb6585cb
       Assigned to: Receipt categorizer
       Status: ✅ Completed
    └── 🤖 Agent: Receipt categorizer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9bd40b6b-2d33-4baf-a93a-82dbdb6585cb                                                                     │
│  Agent: Receipt categorizer                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 94ca4752-4bf0-4fe5-bb50-f623c47b7cfe                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Food


In [ ]:
result.tasks_output[1].raw

'Food'

In [ ]:
result.tasks_output[0].raw

'635'